# Carla speed limit assist with corrective actions and traffic light detection and warning

## Importing of libraries
- **cv2 (OpenCV):** computer vision library for image processing. Used for handling and processing video frames or images.
- **paho.mqtt.client:** client library for the MQTT protocol, this library permit to publish MQTT topics, likely for exchanging data between the CARLA simulator and other systems.
- **ultralytics (YOLO):** provides Python interface for the YOLO (You Only Look Once) object detection framework. It provide object detection tasks, such as identify traffic light, speedLimit.
- **pygame:** library for creating games and multimedia applications, it allows to take input events.
- **screeninfo:** To retrieve monitor specifications, likely for adjusting the display settings of the simulation or output windows.
- **ConfigParser:** For reading configuration files, potentially used to manage simulation settings.

In [ ]:
import glob
import os
import sys
import cv2
import numpy as np
import paho.mqtt.client as mqtt
from ultralytics import YOLO
from screeninfo import get_monitors
import time
from configparser import ConfigParser
import math
import pygame

try:
    sys.path.append(glob.glob('../../carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass

import carla

The following function determines the dimensions of the largest monitor available on the system. It uses the get_monitors function from the screeninfo library to retrieve information about all connected monitors.

In [ ]:
width_screen = 0
height_screen = 0

def get_screen_dimensions():
    global width_screen, height_screen
    monitors = get_monitors()
    for monitor in monitors:
        if width_screen < monitor.width:
            width_screen = monitor.width
            height_screen = monitor.height

get_screen_dimensions()
print(f"Larghezza dello schermo più grande: {width_screen}")
print(f"Altezza dello schermo più grande: {height_screen}")

Larghezza dello schermo più grande: 5120
Altezza dello schermo più grande: 1440


## Broker MQTT configuration
This code sets up an MQTT client to securely connect to a broker using the HiveMQ Cloud service and send messages to specific topics, in base of the event occurred

In [119]:
BROKER = "cd027bc56d0e4f84a8cd8c7558775d7b.s1.eu.hivemq.cloud"
PORT = 8883    

clientMQTT = mqtt.Client()
clientMQTT.username_pw_set("andrea", "Password123")
clientMQTT.tls_set(tls_version=mqtt.ssl.PROTOCOL_TLS)
clientMQTT.connect(BROKER, PORT, 60)
def sendEventToBroker(topic, message):
    try:
        clientMQTT.publish(topic, message)
    except Exception as e:
        pass

c:\Users\andrea.bedei2\AppData\Local\anaconda3\envs\carla-env\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  after removing the cwd from sys.path.


## Client configuration

In [ ]:
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
client.load_world('Town02')
world = client.get_world()
spectator = world.get_spectator()

## Utility functions used in this notebook
- **spawn_vehicle:** Adds a vehicle to the simulation.
  - Picks a vehicle type from the blueprint library (pattern).
  - Selects a spawn point on the map (spawn_index).
  - Places the vehicle at that location.
- **spawn_camera** Adds a camera to the simulation.
  - Sets the camera size, field of view (foV), and update speed.
  - Places the camera at a specific position and angle.
  - Attaches the camera to a vehicle.
- **IMAGE_SIZE** The dimension of the picture taken by the camera, except for the driver.



In [ ]:
IMAGE_SIZE = 640
correct_map = False

def spawn_vehicle(vehicle_index=0, spawn_index=0, pattern='vehicle.*'):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

if world.get_map().name == "Town02" or world.get_map().name == "Town01":
    correct_map = True

def spawn_camera(attach_to=None, sensor_tick=0, transform=carla.Transform(carla.Location(x=1, z=1.8), carla.Rotation(pitch=5, yaw=35)), width=IMAGE_SIZE, height=IMAGE_SIZE, foV=50, exposure_mode='auto', expousure_compensation=0):
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', str(width))
    camera_bp.set_attribute('image_size_y', str(height))
    camera_bp.set_attribute('fov', str(foV))
    camera_bp.set_attribute('sensor_tick', str(sensor_tick))
    camera_bp.set_attribute('exposure_mode', exposure_mode)
    camera_bp.set_attribute('exposure_compensation', str(expousure_compensation))  
    camera = world.spawn_actor(camera_bp, transform, attach_to=attach_to)
    return camera

## Training YOLOv8 Models for Object Detection

### Overview
We have trained the **YOLOv8** model multiple times to detect specific objects with two categories:

1. **Speed Limits**:
    - **First Training**: Using images of the speed limit signs without any background.
    - **Second Training**: Using images of speed limit signs in real-world environments, with significant background clutter for object detection.

2. **Traffic Lights**:
    - **First Training**: Using images of the traffic lights without any background.
    - **Second Training**: Using images of traffic lights in real-world environments, with significant background clutter for object detection.

At the end, we will have **two YOLOv8 models**:
- One dedicated to **speed limits**.
- One dedicated to **traffic lights**.

### Dataset
The datasets used for training were sourced from the following site:  
[Roboflow Universe](https://universe.roboflow.com/)

### Base Code for Downloading and Training the Model
Below is the base code to download the dataset and train the YOLOv8 model: (see training.ipynb)


## Training Model Results
<img src="runsSpeed/detect/train/confusion_matrix.png" alt="Descrizione dell'immagine" width="700">
<img src="runsFinLight/detect/train/confusion_matrix.png" alt="Descrizione dell'immagine" width="700">





## Load Pretrained Models

We load the pretrained weights of yolo model in order to use them for the predictions with GPU.

In [ ]:
model = YOLO("runsSpeed/detect/train/weights/best.pt")
modelTrafficLight = YOLO("runsFinLight/detect/train/weights/best.pt")
modelTrafficLightOver = YOLO("runsUlt/detect/train7/weights/best.pt")
model.to("cuda:0")
modelTrafficLight.to("cuda:0")
modelTrafficLightOver.to("cuda:0")

## Set weather conditions
- Set the worst weather condition or the best weather condition, to set night condition comment the second cell.

In [124]:
weather = world.get_weather()

In [125]:
weather.sun_azimuth_angle = 0
weather.sun_altitude_angle = -90
weather.cloudiness = 100
weather.precipitation = 100
weather.precipitation_deposits = 100
weather.wind_intensity = 100
weather.fog_density = 70
weather.fog_distance = 10

In [ ]:
weather.sun_azimuth_angle = 0 
weather.sun_altitude_angle = 90
weather.cloudiness = 20
weather.precipitation = 0
weather.precipitation_deposits = 0
weather.wind_intensity = 0
weather.fog_density = 30
weather.fog_distance = 100

In [127]:
world.set_weather(weather)

## Color utility functions
- **colorSpeedLimit**: Change the letters color in the view.
- **calcColor**: Change the letters color of the traffic light

In [128]:
def colorSpeedLimit(current_speed, speed_limit):
    if speed_limit == None or int(current_speed) <= int(speed_limit):
        return (0, 255, 0) 
    else:
        return (0, 0, 255)

In [129]:
def calcColor(traffic_light):
    if traffic_light == "red":
        return (0, 0, 255)
    elif traffic_light == "yellow":
        return (0, 255, 255)
    elif traffic_light == "green":
        return (0, 255, 0)
    else:
        return (255, 255, 255)

## Sign_camera_callback

This function processes an image captured by a camera, analyzes it using our machine learning model (Yolo), and identifies the most likely speed limit sign if detected with high confidence. The results are then sent to an external system.

### Key Steps:

1. **Global Variable:**
   - **`last_speed_limit`:** Stores the last detected speed limit value to avoid redundant notifications.

2. **Image Data Conversion:**
   - Converts the raw image data from the simulator to a NumPy array using `np.frombuffer`.
   - Reshapes the data into its original dimensions, including an alpha channel (RGBA format).
   - Removes the alpha channel to work with the RGB image (`image_bgr`).

3. **Model Prediction:**
   - Passes the RGB image (`image_bgr`) to YOLO for analysis.
   - Extracts the detected class IDs (`class_ids`) and their corresponding confidence scores (`confidences`) from the model’s results.

4. **Identify the Most Likely Speed Limit:**
   - Iterates through detected objects, keeping track of the class ID with the highest confidence score.
   - If the highest confidence exceeds a threshold (`0.87`), identifies the corresponding class name from the `class_names` list.

5. **Extract and Notify Speed Limit:**
   - Attempts to extract the speed limit value from the class name using string splitting (`class_name.split(" ")[2]`).
   - If the detected speed limit is new or different from the previous one, sends an event to an external broker using `sendEventToBroker`.
   - Updates `last_speed_limit` with the detected value.

### Notes:
- The function ensures that only high-confidence detections are processed to reduce false positives.
- It avoids redundant notifications by checking if the new detection differs from the last one.
- Any errors during the string extraction or event sending are silently handled.

This callback is integral to detecting and responding to traffic signs in real-time.


In [ ]:
def traffic_sign_camera_callback(image, class_names):
    global last_speed_limit

    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    image_np = array.reshape((IMAGE_SIZE, IMAGE_SIZE, 4))
    image_bgr = image_np[:, :, :3]

    results = model(image_bgr, verbose=False)
    limit_id = None
    confidence_max = 0
    class_ids = results[0].boxes.cls.cpu().numpy()
    confidences = results[0].boxes.conf.cpu().numpy()
    
    for class_id, confidence in zip(class_ids, confidences):
        if confidence > confidence_max:
            limit_id = class_id
            confidence_max = confidence

    if confidence_max > 0.87:
        class_name = class_names[int(limit_id)]
        try:
            if last_speed_limit == None or last_speed_limit != class_name.split(" ")[2]:
                sendEventToBroker("speedLimit", "Detected " + class_name.split(" ")[2])
            last_speed_limit = class_name.split(" ")[2]
        except:
            pass

## View_camera_callback:

This function processes an image captured by the camera dedicated to the driver and updates the global variables used for video output.

1. **Global Variables:**
   - `video_output`: A global variable used to store the processed image data for visualization or further use.

In [ ]:
def view_camera_callback(image):
    global video_output
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    image_np = array.reshape((height_screen, width_screen, 4))
    video_output = image_np[:, :, :3]

## Camera_trafficlight_callback:

This function processes an image from a camera to detect the current state of a traffic light. It enhances the image, analyzes it with YOLO, updates global variables, and sends notifications to the MQTT broker when the state changes.

### Key Steps:

1. **Global Variable:**
   - **`traffic_light`:** Stores the detected traffic light state (`"red"`, `"yellow"`, `"green"`, or `"Not detected"`).

2. **Image Data Conversion:**
   - Converts the raw image data from the camera into a NumPy array using `np.frombuffer`.
   - Reshapes the array into an image matrix of size `IMAGE_SIZE x IMAGE_SIZE` with 4 channels (BGRA format).
   - Extracts the RGB portion (`image_bgr`) for further processing.

3. **Traffic Light Detection:**
   - Passes the processed image (`resized_image`) to the traffic light detection model (YOLO).
   - Retrieves the detected class IDs (`class_ids`) and their confidence scores (`confidences`).
   - If no traffic lights are detected, sets `traffic_light` to `"Not detected"`.

4. **Identify the Most Likely Traffic Light State:**
   - Iterates through the detected objects, identifying the class ID with the highest confidence score.
   - Determines the traffic light state (`"red"`, `"yellow"`, or `"green"`) based on the class name.
   - Applies confidence thresholds to reduce false positives:
     - **High confidence (`>0.55`)** for `"red"` and `"green"`.
     - **Moderate confidence (`>0.45`)** for `"yellow"`.

5. **State Update and Notification:**
   - If the detected traffic light state differs from the previous state stored in `traffic_light`, updates `traffic_light`.
   - Sends a notification to the MQTT broker using `sendEventToBroker` with the new traffic light state.

### Notes:
- The function ensures that only high-confidence detections trigger updates.
- It differentiates between colors with specific thresholds to improve accuracy.
- Any significant state changes are notified to MQTT broker.


In [ ]:
def camera_trafficlight_callback(image, class_names_trafficLight):
    global traffic_light
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    image_np = array.reshape((IMAGE_SIZE, IMAGE_SIZE, 4))
    image_bgr = image_np[:, :, :3].copy()
    resized_image = image_bgr.copy()

    results = modelTrafficLight(resized_image, verbose=False)
    trafficLight_id = None
    confidence_max = 0
    class_ids = results[0].boxes.cls.cpu().numpy()
    confidences = results[0].boxes.conf.cpu().numpy()

    if class_ids.size == 0:
        traffic_light = "Not detected"
    else:
        for class_id, confidence in zip(class_ids, confidences):
            if confidence > confidence_max:
                trafficLight_id = class_id
                confidence_max = confidence

        if traffic_light != None and class_names_trafficLight[int(trafficLight_id)] == "yellow" and traffic_light == "red":
            class_name = "red"
        else:
            class_name = class_names_trafficLight[int(trafficLight_id)]

        if confidence_max > 0.55 and class_name=="red" or confidence_max > 0.55 and class_name=="green" or confidence_max > 0.45 and class_name=="yellow":
            if traffic_light != class_name:
                traffic_light = class_name
                sendEventToBroker("TrafficLight", "Detected trafficlight " + class_name)

## Red_over_camera_callback:

This function operates similarly to `red_over_camera_callback(image)` but focuses specifically on detecting and reacting to **red traffic lights**, modifying a global flag (`red_over`) based on the detection results:

1. **Global Variables:**
   - `last_analysis_time_trafficLight_red`: Tracks the time of the last analysis for red traffic lights to ensure periodic updates.
   - `red_over`: A flag indicating whether a red light condition has been detected and action of stopping should be enforced.

2. **Traffic Light Detection:**
   - Passes the processed image to the `modelTrafficLight`, which detects traffic light states and outputs class IDs (`class_ids`) and their confidence scores (`confidences`).
   - Handles cases where no traffic lights are detected by resetting `red_over` to `False`.

3. **Red Light Detection Logic:**
   - Iterates through detected objects, identifying the class ID with the highest confidence score.
   - If the detected class is `"red"` and the confidence score exceeds `0.65`: Sets `red_over` to `True` **unless the vehicle is in reverse (`gear != "Reverse"`)** or another condition prevents it.
   - If the detected class is `"green"` or `"yellow"` and the confidence score exceeds `0.43`: Resets `red_over` to `False`, allowing movement.

In [ ]:
def red_over_camera_callback(image, class_names_trafficLight):
    global red_over, video_output2
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    image_np = array.reshape((IMAGE_SIZE, IMAGE_SIZE, 4))
    image_bgr = image_np[:, :, :3].copy()
    resized_image = image_bgr.copy()
    video_output2 = resized_image

    results = modelTrafficLightOver(resized_image, verbose=False)

    trafficLight_id = None
    confidence_max = 0

    class_ids = results[0].boxes.cls.cpu().numpy()
    confidences = results[0].boxes.conf.cpu().numpy()

    if class_ids.size != 0:
        for class_id, confidence in zip(class_ids, confidences):
            if confidence > confidence_max:
                trafficLight_id = class_id
                confidence_max = confidence

        class_name = class_names_trafficLight[int(trafficLight_id)]
        if class_name=="red" and confidence_max > 0.489:
            if(gear != "Reverse" and not red_over): # type: ignore
                red_over = True

## Hazard Lights Control
These functions manage the activation and deactivation of hazard lights after a red over. Hazard lights indicate an emergency or warning scenario.

In [134]:
def activate_hazard_lights(vehicle):
    global hazard_lights_active, hazard_lights_start_time
    hazard_lights_active = True
    hazard_lights_start_time = time.time()
    vehicle.set_light_state(carla.VehicleLightState(carla.VehicleLightState.RightBlinker | carla.VehicleLightState.LeftBlinker))

def deactivate_hazard_lights(vehicle):
    global hazard_lights_active
    hazard_lights_active = False
    vehicle.set_light_state(carla.VehicleLightState.NONE)

## PrintTextOnScreen
Utility function to print words on the screen.

In [ ]:
def print_text_on_screen(temp_frame, text, position, color):
    cv2.putText(
        temp_frame,
        text,
        position,
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        color,
        4,
        cv2.LINE_AA,
    )

## Check_red_over:
This function checks if the red over was already actuated, in this case it won't do anything otherwise it will  send the message to the MQTT broker.

In [ ]:
def check_red_over(text_red_val):
    if red_over:
        if text_red_val != "You ran a red light":
            sendEventToBroker("TrafficLightViolation", "Red light violation")
        return "You ran a red light"
    else:
        return ""
    

## init_joystick:
This function initializes the joystick, configures the controls for a racing wheel (G29), and stores joystick input values for later use in controlling the vehicle.
1. **Joystick Setup:**
   - Initializes the first joystick (`Joystick(0)`) and sets it up for use.
   - Reads the configuration file `wheel_config.ini` to retrieve control mappings for various inputs.

2. **Control Mappings:**
   - Parses the configuration file to obtain the indices for the following controls:
     - **`steer_idx`:** Steering wheel control index.
     - **`throttle_idx`:** Throttle control index.
     - **`brake_idx`:** Brake control index.
     - **`reverse_idx`:** Reverse control index.
     - **`handbrake_idx`:** Handbrake control index.

3. **Joystick Inputs:**
   - Retrieves the number of axes on the joystick (`numAxes`).
   - Creates a list of joystick inputs (`jsInputs`), storing the current values for each axis.

In [ ]:
def init_joistick():
    global joystick, joystick_count, parser, steer_idx, throttle_idx, brake_idx, reverse_idx, handbrake_idx, numAxes, jsInputs
    pygame.joystick.init()
    joystick_count = pygame.joystick.get_count()
    if joystick_count > 1:
        raise ValueError("Please Connect Just One Joystick")

    joystick = pygame.joystick.Joystick(0)
    joystick.init()

    parser = ConfigParser()

    parser.read('.\wheel_config.ini')
    steer_idx = int(
    parser.get('G29 Racing Wheel', 'steering_wheel'))
    throttle_idx = int(
    parser.get('G29 Racing Wheel', 'throttle'))
    brake_idx = int(parser.get('G29 Racing Wheel', 'brake'))
    reverse_idx = int(parser.get('G29 Racing Wheel', 'reverse'))
    handbrake_idx = int(
    parser.get('G29 Racing Wheel', 'handbrake'))

    numAxes = joystick.get_numaxes()
    jsInputs = [float(joystick.get_axis(i)) for i in range(numAxes)]

## Main Control Loop for CARLA

This section of the code sets up the environment, initializes variables, and runs the main control loop for manual control of a CARLA simulation vehicle. The loop handles vehicle control based on joystick or keyboard inputs, processes events, and manages the vehicle's state, such as gear, speed, and hazard light status.

### Steps:

1. **Pygame Window Setup:**
   - Creates a Pygame window.
   - Calls `init_joistick()` to initialize the joystick for controlling the vehicle.

2. **Control Variables:**
   - Defines incremental values for steering and throttle adjustments (`steer_increment`, `throttle_increment`).
   - Sets the initial values for vehicle controls such as `steer`, `throttle`, `brake`, and `gear`.
   - Initialization of: flags for speed control, autopilot activation, red light detection, and other features.

3. **Proportional Controller Constants:**
   - Defines constants for controlling throttle (`KP_THROTTLE`), brake (`KP_BRAKE`), and dead zone (`DEAD_ZONE`), alongside minimum values for throttle and brake.

4. **Video Frame Initialization:**
   - Initializes a `video_output` array to store video frames captured by the vehicle's camera.

5. **Speed and Traffic Light Tracking:**
   - Initializes variables to track the vehicle's speed (`car_speed`) and traffic light state (`traffic_light`).

6. **Model Class Names:**
   - Loads class names for object detection models (`model.names` for object detection and `modelTrafficLight.names` for traffic light detection).

7. **Vehicle and Camera Setup:**
   - Spawns a vehicle (`vehicle`) and attaches cameras for different purposes, such as speed limit detection and traffic light detection.
   - Configures cameras with specific parameters (e.g., resolution, field of view) and attaches callbacks to handle image processing.

8. **Event Loop:**
    - Runs the main loop, handling joystick, keyboard, and game events. The loop:
      - Retrieves and processes vehicle velocity.
      - Displays various vehicle status overlays on the screen (e.g., current speed, gear, traffic light state).
      - Handles user inputs (e.g., speed control, autopilot toggle, red light detection) via keyboard and joystick buttons.
      - Applies proportional control to adjust throttle and brake based on speed limits.
      - Activates hazard lights and brakes if the red light detection feature is triggered.

9. **Control Application:**
    - At each frame, the vehicle's control (throttle, brake, steer, reverse) is updated based on input values.
    - If hazard lights are active for more than 2 seconds, they are deactivated.

10. **Clean Up:**
    - Finally, ensures proper cleanup of resources, including destroying windows, disconnecting the MQTT client, and removing the vehicle and camera objects.

### Event Handling:
- **Joystick and Keyboard Inputs:** The loop checks for joystick button presses and keyboard key events to toggle features like speed control, autopilot, red light detection, and gear changes.
- **Throttle and Brake Control:** Speed control logic adjusts throttle and brake values based on the current speed and speed limits.
- **Steering Control:** Steering is controlled using joystick input or keyboard keys (A/D for left/right).

In [ ]:
displayCommand = pygame.display.set_mode((200, 100))
pygame.display.set_caption("Manual Control CARLA")
init_joistick()

hazard_lights_active = False
hazard_lights_start_time = 0

# Define control variables and increments
steer_increment = 0.02  # Increment for steering adjustments
throttle_increment = 0.1  # Increment for throttle adjustments
steer = 0.0  # Current steering value
throttle = 0.0  # Current throttle value
brake = 0.0  # Current brake value
global gear  # Define a global gear variable
gear = "Drive"  # Initial gear state
speed_control_activate = False  # Flag for speed control activation
auto_pilot_activate = False  # Flag for autopilot activation
red_over_activate = False  # Flag for red light detection activation
red_over = False  # Red light violation flag
global control # Set the car control
text_red_val = ""

# Constants for proportional controller
KP_THROTTLE = 0.15  # Proportional gain for acceleration
KP_BRAKE = 0.02  # Proportional gain for braking
DEAD_ZONE = 3.0  # Dead zone around the speed limit
MIN_THROTTLE = 0.2  # Minimum throttle value
MIN_BRAKE = 0.1  # Minimum brake value

# Initialize the output video frame
video_output = np.zeros((width_screen, height_screen, 3), dtype=np.uint8)

# Variables for speed and traffic light tracking
last_speed_limit = None  # Last detected speed limit
car_speed = 0  # Current vehicle speed
traffic_light = "Not detected"  # Current traffic light state

# Load class names for object detection models
class_names = model.names  # Object detection model names
class_names_trafficLight = modelTrafficLight.names  # Traffic light model names

# Spawn the simulated vehicle
vehicle = spawn_vehicle()

# Checks the sun altitude and if it is dark turns on the lights
sun_altitude = weather.sun_altitude_angle
if sun_altitude < 10:
    vehicle.set_light_state(carla.VehicleLightState.HighBeam)
else: 
    vehicle.set_light_state(carla.VehicleLightState.NONE)


# Attach and configure cameras for various purposes
speed_limit_camera = spawn_camera(attach_to=vehicle, sensor_tick=0.2) # Camera focused on speedlimit
speed_limit_camera.listen(lambda image: traffic_sign_camera_callback(image, class_names))

camera_view = spawn_camera(
    attach_to=vehicle, 
    sensor_tick=0.0,
    transform=carla.Transform(carla.Location(x=1, z=1.5), carla.Rotation(pitch=0, yaw=0)), 
    width=width_screen, 
    height=height_screen, 
    foV=115
)  # Driver's perspective camera
camera_view.listen(lambda image: view_camera_callback(image))

if correct_map:
    trafficLight_camera = spawn_camera(
        attach_to=vehicle, 
        sensor_tick=0.25,
        transform=carla.Transform(carla.Location(x=0.3, y=1, z=1.2), carla.Rotation(pitch=10, yaw=5)), 
        width=IMAGE_SIZE, 
        height=IMAGE_SIZE, 
        foV=35,
    ) # Camera focused on trafficlight
    trafficLight_camera.listen(lambda image: camera_trafficlight_callback(image, class_names_trafficLight))
    
    trafficLight_red_camera = spawn_camera(
        attach_to=vehicle,
        sensor_tick=0.10,
        transform=carla.Transform(carla.Location(x=-1, y=0, z=2), carla.Rotation(pitch=-5, yaw=5)), 
        width=IMAGE_SIZE, 
        height=IMAGE_SIZE, 
        foV=120
    ) # Camera focused on red over
    trafficLight_red_camera.listen(lambda image: red_over_camera_callback(image, class_names_trafficLight))

# Create an OpenCV window for displaying results
cv2.namedWindow('RGB Camera', cv2.WINDOW_FULLSCREEN)

try:
    clock = pygame.time.Clock()  # Pygame clock for controlling loop rate
    event_timer = 0  # Timer for event processing
    EVENT_RATE = 100  # Event handling interval in milliseconds

    while True:
        time_hazard = time.time()
        jsInputs = [float(joystick.get_axis(i)) for i in range(numAxes)]

        # Get the car velocity.
        car_velocity = vehicle.get_velocity()
        car_speed = 3.6 * (car_velocity.x**2 + car_velocity.y**2 + car_velocity.z**2)**0.5

        # Add text overlays to the output frame
        temp_frame = video_output.copy()
        print_text_on_screen(temp_frame, f"Last Speed Limit: {last_speed_limit}", (10, 30), (0, 255, 0))
        print_text_on_screen(temp_frame, f"Current speed: {car_speed:.0f}", (10, 70), colorSpeedLimit(car_speed, last_speed_limit))
        print_text_on_screen(temp_frame,  f"Traffic light: {traffic_light}", (10, 110), calcColor(traffic_light))
        print_text_on_screen(temp_frame, f"(LT RT) Gear: {gear}", (10, 150), (255, 255, 255))
        print_text_on_screen(temp_frame,  f"B Autopilot: {auto_pilot_activate}", (10, 190), (255, 255, 255))
        print_text_on_screen(temp_frame, f"X SpeedControl: {speed_control_activate}", (10, 230), (255, 255, 255))
        print_text_on_screen(temp_frame,  f"Y Red Over Automatic Brake: {red_over_activate}", (10, 270), (255, 255, 255))
        # If the speed is 0, red over is turned off.
        if car_speed<=1:
            red_over = False
        text_red_val = check_red_over(text_red_val)
        print_text_on_screen(temp_frame,  text_red_val, (10, 310), (0, 0, 255))

        # Show the frame with overlays
        cv2.imshow('RGB Camera', temp_frame)

        # Break the loop on 'q' key press
        if cv2.waitKey(1) == ord('q'):
            break

        # Handle Pygame events periodically
        current_time = pygame.time.get_ticks()
        if current_time - event_timer > EVENT_RATE:
            event_timer = current_time
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    break
                elif event.type == pygame.KEYUP:  # Handle key releases
                    if event.key == pygame.K_x:  # Toggle speed control
                        speed_control_activate = not speed_control_activate
                    if event.key == pygame.K_b:  # Enable autopilot
                        auto_pilot_activate = not auto_pilot_activate
                        vehicle.set_autopilot(auto_pilot_activate)
                    if event.key == pygame.K_y:  # Toggle red light detection
                        red_over_activate = not red_over_activate
                    if event.key == pygame.K_r:  # Reverse gear
                        gear = "Reverse"
                        red_over = False
                    elif event.key == pygame.K_l:  # Forward gear
                        gear = "Drive"
                elif event.type == pygame.JOYBUTTONDOWN:
                    if event.button == 1: # Toggle speed control
                        speed_control_activate = not speed_control_activate
                    elif event.button == 2: # Enable autopilot
                        auto_pilot_activate = not auto_pilot_activate
                        vehicle.set_autopilot(auto_pilot_activate)
                    elif event.button == 3: # Toggle red light detection
                        red_over_activate = not red_over_activate
                    elif event.button == 6: # Reverse gear
                        gear = "Reverse"
                        red_over = False
                    elif event.button == 7: # Forward gear
                        gear = "Drive"


            keys = pygame.key.get_pressed()  # Get pressed keys

            K2 = 1.6
            # Fix bugs, at the beginning js returns only 0.0 value
            if jsInputs[throttle_idx] == 0.0:
                throttleCmd = 0
            else:
                throttleCmd =  K2 + (2.05 * math.log10(0.7 * jsInputs[throttle_idx] + 1.4) - 1.2) / 0.92
                if throttleCmd <= 0:
                    throttleCmd = 0
                elif throttleCmd > 1:
                    throttleCmd = 1

            # Handle speed control logic and throttle
            if speed_control_activate and last_speed_limit and (jsInputs[throttle_idx]>-0.95 or jsInputs[throttle_idx]>0.0):
                # Calculate speed error relative to the limit
                error = float(last_speed_limit) - float(car_speed) + 3
                if error < -DEAD_ZONE:
                    throttle = 0
                    brake = max(KP_BRAKE * abs(error), MIN_BRAKE)
                else:
                    throttle = min(max(KP_THROTTLE * error, MIN_THROTTLE), throttleCmd)
                    brake = 0
            elif speed_control_activate and last_speed_limit and keys[pygame.K_w]:
                # Calculate speed error relative to the limit
                error = float(last_speed_limit) - float(car_speed) + 3
                if error < -DEAD_ZONE:
                    throttle = 0
                    brake = max(KP_BRAKE * abs(error), MIN_BRAKE)
                else:
                    throttle = max(KP_THROTTLE * error, MIN_THROTTLE)
                    brake = 0
            elif keys[pygame.K_w]:  # Manual throttle control
                throttle = min(throttle + throttle_increment, 1)
                brake = 0
            elif throttleCmd>-0.95:
                throttle = throttleCmd
                brake=0
            else:
                throttle = 0
                brake = 0

            # Handle brake logic
            brakeCmd = 1.6 + (2.05 * math.log10(0.7 * jsInputs[brake_idx] + 1.4) - 1.2) / 0.92
            if brakeCmd <= 0:
                brakeCmd = 0
            elif brakeCmd > 1:
                brakeCmd = 1
            
            if brakeCmd>0.1:
                brake = brakeCmd
                throttle = 0
            elif keys[pygame.K_s]:  # Manual brake control
                brake = min(brake + throttle_increment * 4, 1)
                throttle = 0

            # Check it the brake will be activated due to a red over.
            if red_over and int(car_speed) <= 50 and gear != "Reverse" and red_over_activate:
                activate_hazard_lights(vehicle)
                brake = 1
                throttle = 0
            
            # Turn off the red over if it is not activate.
            if not red_over_activate:
                red_over = False
            

            # Handle steering controls
            if keys[pygame.K_a]:  # Steer left
                steer = max(steer - steer_increment, -1)
            elif keys[pygame.K_d]:  # Steer right
                steer = min(steer + steer_increment, 1)
            else:
                if joystick_count == 1:
                    K1 = 1.0  
                    steer = K1 * math.tan(1.1 * jsInputs[steer_idx])
                else:
                    steer = steer * 0.9  # Gradually return to center

            if hazard_lights_active and (time_hazard - hazard_lights_start_time > 2):
                deactivate_hazard_lights(vehicle)

            control = carla.VehicleControl()
            control.throttle = throttle
            control.brake = brake
            control.steer = steer
            control.reverse = (gear == "Reverse")
            vehicle.apply_control(control)

        clock.tick(40)  # Maintain a 40 FPS loop rate
finally:
    # Clean up resources and connections
    cv2.destroyAllWindows()
    pygame.quit()
    speed_limit_camera.destroy()
    trafficLight_camera.destroy()
    trafficLight_red_camera.destroy()
    camera_view.destroy()
    vehicle.destroy()
    clientMQTT.disconnect()
